# Lista 1
W tym pliku znajdują się rozwiąznia zadań do listy 1. oraz komentarze i odpowiedzi do pytań zawartych w poleceniu. Dodatkowo do zadania 1. program w języku C znajduje się w pliku *zad1.c*.  
Michał Ilski, Obliczenia naukowe 2020.

## Zad. 1

Opis: W zadaniu należy wyznaczyć wartości maszynowego epsilona, eta oraz maksymalną możliwą wartość dla arytmetyk Float16, Float32, Float64.

Funkcja obliczająca epsilony maszynowe dla danego typu (Float16/Float32/Float64).

In [1]:
function custom_macheps(fl)
    result = fl(1.0)
    while fl(1.0) + fl(result / 2.0) > fl(1.0)
        result /= fl(2.0)
    end
    return result
end

custom_macheps (generic function with 1 method)

In [2]:
print("Machine Epsilon:\n")
print("Float16: Custom macheps: ", custom_macheps(Float16), " | Julia eps: ", eps(Float16), "\n")
print("Float32: Custom macheps: ", custom_macheps(Float32), " | Julia eps: ", eps(Float32), "\n")
print("Float64: Custom macheps: ", custom_macheps(Float64), " | Julia eps: ", eps(Float64), "\n")

Machine Epsilon:
Float16: Custom macheps: 0.000977 | Julia eps: 0.000977
Float32: Custom macheps: 1.1920929e-7 | Julia eps: 1.1920929e-7
Float64: Custom macheps: 2.220446049250313e-16 | Julia eps: 2.220446049250313e-16


float.h dla języka C (gcc 9.3.0) zawiera wartości epsilon:
- Float32 (float) machine epsilon: 1.1920929e-07
- Float64 (double) machine epsilon: 2.220446049250313e-16

Wyniki i wnioski:  
Zarówno w przypadku napisanej funkcji jak i pochodzącej z języka Julia, dostajemy te same wyniki. Podobnie w języku C z pominięciem Float16. Dla wyższej precyzji jesteśmy w stanie zapisać więcej bitów,  czyli w szczególności otrzymujemy mniejszy epsilon maszynowy.

1) Jaki związek ma macheps z precyzją arytmetyki?  

Wiemy, że precyzja e=0.5\*B^(1-t), gdzie B - baza, t - długość mantysy+1. Dla systemu binarnego mamy e=2^(-t). Weźmy przykładowo arytmetykę Float32.

In [3]:
print(Float32(1.0), "(dec) = ",bitstring(Float32(1.0)), "(bin)\n")

1.0(dec) = 00111111100000000000000000000000(bin)


Następnie dodaję 2^(-24) - jest to 2 razy mniejsza wartość niż najmniejsza wartość jaką mogę realnie dodać (czyli o 1 rząd mniejsza).

In [4]:
print(bitstring(Float32(1.0)+Float32(2^(-24))),"\n")

00111111100000000000000000000000


A tutaj 1+macheps:

In [5]:
print(bitstring(Float32(1.0)+eps(Float32)),"\n")

00111111100000000000000000000001


Czyli 2\*e = macheps.

In [6]:
eps(Float32) == Float32(2.0)*Float32(2^(-24))

true

In [7]:
function custom_eta(fl)
    result = fl(1.0)
    while fl(result / 2.0) > fl(0.0)
        result /= fl(2.0)
    end
    return result
end

custom_eta (generic function with 1 method)

In [8]:
print("Eta:\n")
print("Float16: Custom eta: ", custom_eta(Float16), " | Julia eta: ", nextfloat(Float16(0.0)), "\n")
print("Float32: Custom eta: ", custom_eta(Float32), " | Julia eta: ", nextfloat(Float32(0.0)), "\n")
print("Float64: Custom eta: ", custom_eta(Float64), " | Julia eta: ", nextfloat(Float64(0.0)), "\n")

Eta:
Float16: Custom eta: 6.0e-8 | Julia eta: 6.0e-8
Float32: Custom eta: 1.0e-45 | Julia eta: 1.0e-45
Float64: Custom eta: 5.0e-324 | Julia eta: 5.0e-324


Wyniki i wnioski:  
Funkcja z zadania jak i z języka Julia zwracają te same wyniki. Podobnie jak wyżej, większa precyzja pozwala zapisać więcej liczb zmiennoprzecinkowych, co przekłada się na mniejszą liczbę następną po 0.0.

2) Jaki związek ma liczba eta z liczbą MIN_sub?

Obliczmy ze wzoru MIN_sub=2^(-(t-1))\*2^(c_min) przykładowo dla Float32 (t=24).

In [9]:
print((Float32(2.0^(-(23.0)))*Float32(2.0^-126.0)), '\n')

1.0e-45


Jak widać wartości te są identyczne.

3) Co zwracają floatmin(Float32) i floatmin(Float64) i jaki jest związek z MIN_nor?

In [10]:
print("Float min:\n")
print("Float32: ", floatmin(Float32), "\n")
print("Float64: ", floatmin(Float64), "\n")

Float min:
Float32: 1.1754944e-38
Float64: 2.2250738585072014e-308


Funkcje floatmin zwracają najmniejsze znormalizowane wartości dla poszczególnych arytmetyk. Zgodnie ze wzorem MIN_nor = 2^(c_min). Dla Float32:

In [11]:
print(Float32(2.0^(-126.0)), '\n')

1.1754944e-38


Stąd jak widać MIN_nor = floatmin dla danej arytmetyki.

Funkcja zwaracająca maksymalną wartość, jaką można zapisać w danej precyzji (Float16/Float32/Float64).

In [12]:
function custom_max(fl)
    result = fl(1.0)
    while !isinf(fl(result) * fl(2.0))
        result *= fl(2.0)
    end
    #here we want to multiply our result by largest number which will give result under inf
    result *= fl(2.0)-eps(fl)
    return result
end

custom_max (generic function with 1 method)

In [13]:
print("Max float:\n")
print("Float16: Custom max: ", custom_max(Float16), " | Julia max: ", floatmax(Float16(0.0)), "\n")
print("Float32: Custom max: ", custom_max(Float32), " | Julia max: ", floatmax(Float32(0.0)), "\n")
print("Float64: Custom max: ", custom_max(Float64), " | Julia max: ", floatmax(Float64(0.0)), "\n")

Max float:
Float16: Custom max: 6.55e4 | Julia max: 6.55e4
Float32: Custom max: 3.4028235e38 | Julia max: 3.4028235e38
Float64: Custom max: 1.7976931348623157e308 | Julia max: 1.7976931348623157e308


float.h dla języka C zawiera wartość max:
- Float32 max: 3.4028235e+38
- Float62 max: 1.797693134862316e+308

Wyniki i wnioski:  
Podobnie jak w przypadku epsilon, wartości funkcji z zadania, z języka Julia oraz C są identyczne dla odpowiednich typów. Dla większych precyzji możemy zapisać więcej liczb, dlatego wartości maksymalne są odpowiednio większe.

## Zad. 2

Opis: W zadaniu należy sprawdzić poprawność alternatywnej metody znajdowania epsilona maszynowego.

In [14]:
function kahan_method(fl)
    return fl(3.0) * (fl(4.0 / 3.0) - fl(1.0)) - fl(1.0)
end

kahan_method (generic function with 1 method)

In [15]:
print("Machine Epsilon:\n")
print("Float16: Kahan method: ", kahan_method(Float16), " | Julia eps: ", eps(Float16), "\n")
print("Float32: Kahan method: ", kahan_method(Float32), " | Julia eps: ", eps(Float32), "\n")
print("Float64: Kahan method: ", kahan_method(Float64), " | Julia eps: ", eps(Float64), "\n")

Machine Epsilon:
Float16: Kahan method: -0.000977 | Julia eps: 0.000977
Float32: Kahan method: 1.1920929e-7 | Julia eps: 1.1920929e-7
Float64: Kahan method: -2.220446049250313e-16 | Julia eps: 2.220446049250313e-16


Wyniki i wnioski:  
Metoda Kahana działa poprawnie z dokładnością do znaku, zatem może stanowić alternatywę do podjeścia iteracyjnego. Poniższa implementacja naprawia błąd związany ze znakiem.

In [16]:
function kahan_method_abs(fl)
    return abs(fl(3.0) * (fl(4.0 / 3.0) - fl(1.0)) - fl(1.0))
end

kahan_method_abs (generic function with 1 method)

In [17]:
print("Machine Epsilon:\n")
print("Float16: Kahan method with abs: ", kahan_method_abs(Float16), " | Julia eps: ", eps(Float16), "\n")
print("Float32: Kahan method with abs: ", kahan_method_abs(Float32), " | Julia eps: ", eps(Float32), "\n")
print("Float64: Kahan method with abs: ", kahan_method_abs(Float64), " | Julia eps: ", eps(Float64), "\n")

Machine Epsilon:
Float16: Kahan method with abs: 0.000977 | Julia eps: 0.000977
Float32: Kahan method with abs: 1.1920929e-7 | Julia eps: 1.1920929e-7
Float64: Kahan method with abs: 2.220446049250313e-16 | Julia eps: 2.220446049250313e-16


## Zad. 3

Opis: W zadaniu należy sprawdzić rozmieszczenie liczb w arytmetyce Float64 w zależności od zakresu wartości liczb.

Funkcja wyliczająca następną liczbę według wzoru z zadania.

In [18]:
function next_number(previous, k, step)
    return Float64(previous) + Float64(k) * Float64(step)
end

next_number (generic function with 1 method)

Funkcja testująca tezę z zadania.

In [19]:
function test(start, stop, step, secure_limit)
    number = Float64(start)
    k = Float64(1.0)
    counter = 1
    print("Starting from: ", bitstring(number), "\n")
    while next_number(number, k, step) <= Float64(stop) && counter <= secure_limit
        print("nextfloat: ", bitstring(nextfloat(number)))
        number = next_number(number, k, step)
        print(" Bits: ", bitstring(number), "\n")
        counter += 1
    end
end

test (generic function with 1 method)

Wywołanie testu dla przedziału \[1,2\], krok 2^(-52). W celach bezpieczeństwa dodano limit drukowanych rezulatatów (ostatni parametr).

In [20]:
test(Float64(1.0), Float64(2.0), Float64(2.0)^Float64(-52), 20)

Starting from: 0011111111110000000000000000000000000000000000000000000000000000
nextfloat: 0011111111110000000000000000000000000000000000000000000000000001 Bits: 0011111111110000000000000000000000000000000000000000000000000001
nextfloat: 0011111111110000000000000000000000000000000000000000000000000010 Bits: 0011111111110000000000000000000000000000000000000000000000000010
nextfloat: 0011111111110000000000000000000000000000000000000000000000000011 Bits: 0011111111110000000000000000000000000000000000000000000000000011
nextfloat: 0011111111110000000000000000000000000000000000000000000000000100 Bits: 0011111111110000000000000000000000000000000000000000000000000100
nextfloat: 0011111111110000000000000000000000000000000000000000000000000101 Bits: 0011111111110000000000000000000000000000000000000000000000000101
nextfloat: 0011111111110000000000000000000000000000000000000000000000000110 Bits: 0011111111110000000000000000000000000000000000000000000000000110
nextfloat: 001111111111000000000000000

Wyniki i wnioski:  
W powyższym przypadku widzimy, że wartości funkcji next_number jak i wbudowanej funkcji nextfloat są identyczne. W zakresie \[2^0, 2^1\] bit znaku oraz cecha nie ulegają zmianie. W mantysie widać, że każdy następny rezultat to kolejna liczba w zapisie binarnym, co oznacza że w danej precyzji nie da się zrobić mniejszego kroku.

In [21]:
test(Float64(0.5), Float64(1.0), Float64(2.0)^Float64(-53), 20)

Starting from: 0011111111100000000000000000000000000000000000000000000000000000
nextfloat: 0011111111100000000000000000000000000000000000000000000000000001 Bits: 0011111111100000000000000000000000000000000000000000000000000001
nextfloat: 0011111111100000000000000000000000000000000000000000000000000010 Bits: 0011111111100000000000000000000000000000000000000000000000000010
nextfloat: 0011111111100000000000000000000000000000000000000000000000000011 Bits: 0011111111100000000000000000000000000000000000000000000000000011
nextfloat: 0011111111100000000000000000000000000000000000000000000000000100 Bits: 0011111111100000000000000000000000000000000000000000000000000100
nextfloat: 0011111111100000000000000000000000000000000000000000000000000101 Bits: 0011111111100000000000000000000000000000000000000000000000000101
nextfloat: 0011111111100000000000000000000000000000000000000000000000000110 Bits: 0011111111100000000000000000000000000000000000000000000000000110
nextfloat: 001111111110000000000000000

Wyniki i wnioski:  
Otrzymane wyniki są analogiczne do tych z przypadku wyżej. Tutaj jednak mamy mniejszy krok, ponieważ rząd liczb z przedziału \[2^(-1), 2^0\] (w zapisie binarnym) jest niższy co pozwala na dokładniejsze zapisanie danych liczb. 

In [22]:
test(Float64(2.0), Float64(4.0), Float64(2.0)^Float64(-51), 20)

Starting from: 0100000000000000000000000000000000000000000000000000000000000000
nextfloat: 0100000000000000000000000000000000000000000000000000000000000001 Bits: 0100000000000000000000000000000000000000000000000000000000000001
nextfloat: 0100000000000000000000000000000000000000000000000000000000000010 Bits: 0100000000000000000000000000000000000000000000000000000000000010
nextfloat: 0100000000000000000000000000000000000000000000000000000000000011 Bits: 0100000000000000000000000000000000000000000000000000000000000011
nextfloat: 0100000000000000000000000000000000000000000000000000000000000100 Bits: 0100000000000000000000000000000000000000000000000000000000000100
nextfloat: 0100000000000000000000000000000000000000000000000000000000000101 Bits: 0100000000000000000000000000000000000000000000000000000000000101
nextfloat: 0100000000000000000000000000000000000000000000000000000000000110 Bits: 0100000000000000000000000000000000000000000000000000000000000110
nextfloat: 010000000000000000000000000

Wyniki i wnioski:  
Wyniki analogiczne jak wyżej. Tutaj krok jest większy niż w poprzednich przypadkach ze względu na wyższy rząd liczb w przedziale \[2^1, 2^2\]. Podsumowując, wraz ze wzrostem rzędu liczb spada dokładność przybliżeń.

## Zad 4.

Opis: W zadaniu należy pokazać, że w arytmetyce Float64 istnieją liczby x, dla których x*(1/x) != 1.

Funkcja wyszukująca najmniejszą liczbę spełniającą warunek z zadania (od razu znajduje wartość najmniejszą).

In [23]:
function find_number()
    x = Float64(1.0)
    while Float64(x * Float64(1/x)) == Float64(1.0) && x < Float64(2.0)
        x = nextfloat(x)
    end
    return x
end

find_number (generic function with 1 method)

In [24]:
x = find_number()
print("Found: ", x, " Bits: ", bitstring(x))

Found: 1.000000057228997 Bits: 0011111111110000000000000000000000001111010111001011111100101010

Wyniki i wnioski:  
Program w krótkim czasie jest w stanie znaleźć przypadek, w którym podana w zadaniu równość nie zachodzi. Wpływ na to ma błąd zaokrągleń w precyzji Float64, co powoduje problem nieodwracalności dzielenia.

## Zad 5.

Opis: W zadaniu należy policzyć iloczyn skalarny na 4 różne sposoby. Zmienie ulaga kolejność argumentów.

Implementacje funkcji z zadania:

In [25]:
function f_a(x, y, fl)
    S = fl(0.0)
    for i in 1:length(x)
        S += fl(x[i]) * fl(y[i])
    end
    return S
end

f_a (generic function with 1 method)

In [26]:
function f_b(x, y, fl)
    S = fl(0.0)
    for i in 5:-1:1
        S += fl(x[i]) * fl(y[i])
    end
    return S
end

f_b (generic function with 1 method)

In [169]:
function f_c(x, y, fl)
    values = Array{fl,1}(zeros(0))
    for i = 1:length(x)
        push!(values, fl(x[i]) * fl(y[i]))
    end
    
    #positive = values[values .>= 0]
    positive = Array{fl,1}(zeros(0))
    println(positive)
    negative = Array{fl,1}(zeros(0))
    for v in values
        if v >= fl(0.0)
            push!(positive, fl(v))
        else
            push!(negative, fl(v))
        end
    end
    sort!(positive, rev=true)
    #negative = values[values .< 0]
    sort!(negative)
    
    p_sum = n_sum = fl(0.0)
    for i in 1:length(positive)
        p_sum += fl(positive[i])
    end
    for i in 1:length(negative)
        n_sum += fl(negative[i])
    end
    return fl(p_sum) + fl(n_sum)
end

f_c (generic function with 1 method)

In [175]:
function f_d(x, y, fl)
    values = fl[]
    for i = 1:length(x)
        push!(values, fl(x[i]) * fl(y[i]))
    end
    
    positive = fl[]
    negative = fl[]
    for v in values
        if v >= fl(0.0)
            push!(positive, fl(v))
        else
            push!(negative, fl(v))
        end
    end
    #positive = values[values .>= fl(0.0)]
    sort!(positive)
    #negative = values[values .< fl(0.0)]
    sort!(negative, rev=true)
    println(positive)
    
    p_sum = n_sum = fl(0.0)
    for i in 1:length(positive)
        p_sum += fl(positive[i])
    end
    for i in 1:length(negative)
        n_sum += fl(negative[i])
    end
    return fl(p_sum) + fl(n_sum)
end

f_d (generic function with 1 method)

In [176]:
function get_values(fl)
    return [fl(2.718281828),fl(-3.141592654),fl(1.414213562),fl(0.5772156649),fl(0.3010299957)], 
                    [fl(1486.2497),fl(878366.9879),fl(-22.37492),fl(4773714.647),fl(0.000185049)]
end

get_values (generic function with 1 method)

In [177]:
function get_rel_error(val, aprox)
    return abs(val-aprox)/abs(val)
end

get_rel_error (generic function with 1 method)

In [178]:
function get_abs_error(val, aprox)
    return abs(val-aprox)
end

get_abs_error (generic function with 1 method)

Wyniki dla poszczególnych wywołań:

In [179]:
fls = [Float32, Float64]
print("Value = -1.00657107000000e-11\n")
for fl in fls
    x, y = get_values(fl)
    print(fl, ":\n")
    print("a) ", f_a(x,y,fl), 
        " relative error: ", get_rel_error(-1.00657107000000e-11, f_a(x,y,fl)), 
        " absolute error: ", get_abs_error(-1.00657107000000e-11, f_a(x,y,fl)), "\n")
    print("b) ", f_b(x,y,fl),
        " relative error: ", get_rel_error(-1.00657107000000e-11, f_b(x,y,fl)), 
        " absolute error: ", get_abs_error(-1.00657107000000e-11, f_b(x,y,fl)), "\n")
    print("c) ", f_c(x,y,fl),
        " relative error: ", get_rel_error(-1.00657107000000e-11, f_c(x,y,fl)), 
        " absolute error: ", get_abs_error(-1.00657107000000e-11, f_c(x,y,fl)), "\n")
    print("d) ", f_d(x,y,fl),
        " relative error: ", get_rel_error(-1.00657107000000e-11, f_d(x,y,fl)), 
        " absolute error: ", get_abs_error(-1.00657107000000e-11, f_d(x,y,fl)), "\n")
end

Value = -1.00657107000000e-11
Float32:
a) -0.4999443 relative error: 4.9668057661282845e10 absolute error: 0.49994429944939167
b) -0.4543457 relative error: 4.51379655800096e10 absolute error: 0.4543457031149343
Float32[]
Float32[]
Float32[]
c) -0.5 relative error: 4.967359135306107e10 absolute error: 0.4999999999899343
Float32[5.57053f-5, 4040.0457, 2.7554628f6]
Float32[5.57053f-5, 4040.0457, 2.7554628f6]
Float32[5.57053f-5, 4040.0457, 2.7554628f6]
d) -0.5 relative error: 4.967359135306107e10 absolute error: 0.4999999999899343
Float64:
a) 1.0251881368296672e-10 relative error: 11.184955313981627 absolute error: 1.1258452438296672e-10
b) -1.5643308870494366e-10 relative error: 14.541186645165915 absolute error: 1.4636737800494365e-10
Float64[]
Float64[]
Float64[]
c) 0.0 relative error: 1.0 absolute error: 1.00657107e-11
[5.57052996742893e-5, 4040.045551380452, 2.7554628740109736e6]
[5.57052996742893e-5, 4040.045551380452, 2.7554628740109736e6]
[5.57052996742893e-5, 4040.045551380452, 2

Wyniki i wnioski:  
Żadna z powyższych kombinacji "precyzja-kolejność" nie była w stanie dokładnie przybliżyć wartości iloczynu. Kolejność wykonawania działań na liczbach okazuje się mieć duży wpływ na dokładność wyniku. Powodem tego jest błąd zaokrągleń, który zależy od rzędu liczb na których działamy.

## Zad 6.

Opis: W zadaniu należy sprawdzić dokładność 2 różnych algorytmów, służących do rozwiązania tego samego problemu.

Podejście pierwsze:

In [33]:
function f(x)
    return sqrt((x^2)+Float64(1.0))-Float64(1.0)
end

f (generic function with 1 method)

Podejście drugie:

In [34]:
function g(x)
    return (x^2)/((sqrt((x^2)+Float64(1.0))+Float64(1.0)))
end

g (generic function with 1 method)

In [35]:
limit = 25

25

In [36]:
for i = 1:limit
    x = Float64(8^((-1.0)*i))
    print("x = 8^(-", i, ") f(x) = ", f(x), " g(x) = ", g(x), "\n")
end

x = 8^(-1) f(x) = 0.0077822185373186414 g(x) = 0.0077822185373187065
x = 8^(-2) f(x) = 0.00012206286282867573 g(x) = 0.00012206286282875901
x = 8^(-3) f(x) = 1.9073468138230965e-6 g(x) = 1.907346813826566e-6
x = 8^(-4) f(x) = 2.9802321943606103e-8 g(x) = 2.9802321943606116e-8
x = 8^(-5) f(x) = 4.656612873077393e-10 g(x) = 4.6566128719931904e-10
x = 8^(-6) f(x) = 7.275957614183426e-12 g(x) = 7.275957614156956e-12
x = 8^(-7) f(x) = 1.1368683772161603e-13 g(x) = 1.1368683772160957e-13
x = 8^(-8) f(x) = 1.7763568394002505e-15 g(x) = 1.7763568394002489e-15
x = 8^(-9) f(x) = 0.0 g(x) = 2.7755575615628914e-17
x = 8^(-10) f(x) = 0.0 g(x) = 4.336808689942018e-19
x = 8^(-11) f(x) = 0.0 g(x) = 6.776263578034403e-21
x = 8^(-12) f(x) = 0.0 g(x) = 1.0587911840678754e-22
x = 8^(-13) f(x) = 0.0 g(x) = 1.6543612251060553e-24
x = 8^(-14) f(x) = 0.0 g(x) = 2.5849394142282115e-26
x = 8^(-15) f(x) = 0.0 g(x) = 4.0389678347315804e-28
x = 8^(-16) f(x) = 0.0 g(x) = 6.310887241768095e-30
x = 8^(-17) f(x) = 0.0

Wyniki i wnioski:  
Mimo, że z matematycznego punktu widzenia dla zadanego x, funkcje f(x) oraz g(x) powinny dać identyczne wyniki, to w arytmetyce Float64 się one różnią. W rzeczywitości obie funkcje dążą do 0, lecz nigdy go nie osiągają. Oznacza to, że funkcja g(x) poradziła sobie lepiej z przybliżaniem wyniku, ponieważ funkcja f(x) już dla x = 8^(-9) zwróciła wartość 0.0. Wynika to z faktu, że funkcja f(x) zawiera odejmowanie liczb, których wartości w kolejnych iteracjach są coraz bardziej zbliżone, a to powoduje znaczące błędy.

## Zad 7.

Opis: W zadaniu należy obliczyć przybliżoną wartość pochodnej funkcji z podanego wzoru i porównać z wartością obliczoną ze wzoru ogólnego na pochodną tej funkcji.

Dana funkcja:

In [37]:
function f(x)
    return sin(x)+cos(Float64(3.0)*x)
end

f (generic function with 1 method)

Pochodna:

In [38]:
function derivative(x)
    return cos(x) - Float64(3.0)*sin(Float64(3.0)*x)
end

derivative (generic function with 1 method)

Wartość przybliżona pochodnej:

In [39]:
function ap_derivative(x, h)
    return Float64(Float64(f(x+h)-f(x))/h)
end

ap_derivative (generic function with 1 method)

In [40]:
x = Float64(1.0)
for i = 0:54
    h = Float64(2^((-1.0)*i))
    print("x = ", x, " h = 2^(-", i, ") approximate df/dx = ", ap_derivative(x,h), " error = ", abs(derivative(x)-ap_derivative(x,h)), " 1+h = ", Float64(1.0)+h, '\n')
end

x = 1.0 h = 2^(-0) approximate df/dx = 2.0179892252685967 error = 1.9010469435800585 1+h = 2.0
x = 1.0 h = 2^(-1) approximate df/dx = 1.8704413979316472 error = 1.753499116243109 1+h = 1.5
x = 1.0 h = 2^(-2) approximate df/dx = 1.1077870952342974 error = 0.9908448135457593 1+h = 1.25
x = 1.0 h = 2^(-3) approximate df/dx = 0.6232412792975817 error = 0.5062989976090435 1+h = 1.125
x = 1.0 h = 2^(-4) approximate df/dx = 0.3704000662035192 error = 0.253457784514981 1+h = 1.0625
x = 1.0 h = 2^(-5) approximate df/dx = 0.24344307439754687 error = 0.1265007927090087 1+h = 1.03125
x = 1.0 h = 2^(-6) approximate df/dx = 0.18009756330732785 error = 0.0631552816187897 1+h = 1.015625
x = 1.0 h = 2^(-7) approximate df/dx = 0.1484913953710958 error = 0.03154911368255764 1+h = 1.0078125
x = 1.0 h = 2^(-8) approximate df/dx = 0.1327091142805159 error = 0.015766832591977753 1+h = 1.00390625
x = 1.0 h = 2^(-9) approximate df/dx = 0.1248236929407085 error = 0.007881411252170345 1+h = 1.001953125
x = 1.0 h

Wyniki i wnioski:
Błąd przybliżenia wartości pochodnej maleje wraz z malejącą wartością h do pewnego momentu, co jest zgodne z oczekiwaniami. Jednak od h = 2^(-29) zaczyna on rosnąć. Dla liczb o dużej różnicy rzędów tracimy dokładność wyników przez konieczność wyrównywania cech, która prowadzi do utraty cyfr znaczących. Podobny problem dotyczy odejmowania zbliżonych do siebie liczb (czyli f(x+h) - f(x) dla małego h).